# Library

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime
import threading
import time
import json

# Settings

In [2]:
base_url = "https://api.waqi.info"
token = open('waqitoken.txt').read()  # https://aqicn.org/api/
# cities_list = ['Rome', 'London', 'Barcelona']
WAIT_TIME_SECONDS = 10 #800  # 10800 -> 3 hour

In [3]:
with open('city.txt', 'r') as f:
    file = f.readlines()

In [12]:
cities_list = [line.rstrip('\n') for line in file]

# City request 

In [14]:
def city_request(city):
    r = requests.get(base_url + f"/feed/{city}/?token={token}")
    return r

# Make record

In [15]:
def make_record(r):
    """Extracts data from request r and returns a DataFrame."""
    item = r.json()
    rows = []
    
    if item['status'] == 'ok':
        # Date
        try:
            rows.append(datetime.datetime.strptime(item['data']['time']['s'], '%Y-%m-%d %H:%M:%S').date())
        except: 
            rows.append(np.nan)
        # Hour
        try:
            rows.append(datetime.datetime.strptime(item['data']['time']['s'], '%Y-%m-%d %H:%M:%S').time())
        except: 
            rows.append(np.nan)
        # City Name
        try:
            rows.append(item['data']['city']['name'])
        except: 
            rows.append(np.nan)  
        # Latitude
        try:
            rows.append(item['data']['city']['geo'][0])
        except: 
            rows.append(np.nan)
        # Longitude
        try:
            rows.append(item['data']['city']['geo'][1])
        except: 
            rows.append(np.nan)
        # AQI
        try:
            rows.append(item['data']['aqi'])
        except: 
            rows.append(np.nan)
        # Create record
        record = pd.Series(rows, index=['local date','local time', 'city', 'lat', 'lon', 'aqi'])
        # I AQI
        try:
            record = record.append(pd.DataFrame(item['data']['iaqi']).iloc[0])
        except:
            pass
        
        return record
    else:
        return np.nan

# Save Dataframe

In [16]:
def save(df):
    df.to_pickle('data.pkl')

# Load Dataframe

In [17]:
def load():
    return pd.read_pickle('data.pkl')

# Make DataFrame

In [20]:
def make_dataframe(record):
    global df
    if type(record) != float:
        df = pd.concat([pd.DataFrame(record).transpose(),df],axis=0, sort=False, ignore_index=True)
        #save(df)

# Application

In [19]:
def data_collect_application():
    global df, cities_list
    
    try:
        df = load()
    except:
        df = pd.DataFrame()

    for city in cities_list:
        make_dataframe(make_record(city_request(city)))
        
    print('Last update: ' + time.ctime())
    print(df)
    
ticker = threading.Event()
while not ticker.wait(WAIT_TIME_SECONDS):
    data_collect_application()

From cffi callback <function _verify_callback at 0x0000025A694F8948>:
Traceback (most recent call last):
  File "C:\Users\ricca\miniconda3\lib\site-packages\OpenSSL\SSL.py", line 311, in wrapper
    @wraps(callback)
KeyboardInterrupt


SSLError: HTTPSConnectionPool(host='api.waqi.info', port=443): Max retries exceeded with url: /feed/Lanzhou/?token=904f192e05a455031abacf9f6cd3c447b29b2be4 (Caused by SSLError(SSLError("bad handshake: Error([('SSL routines', 'tls_process_server_certificate', 'certificate verify failed')])")))

In [24]:
df = pd.DataFrame()
i=1
for city in cities_list:
    print(i)
    make_dataframe(make_record(city_request(city)))
    i = i+1

1
2
3
4
5
6
7
8


KeyboardInterrupt: 

In [23]:
df

,local date,local time,city,lat,lon,aqi,co,h,no2,o3,...,w,dew,wg,d,wd,r,uvi,pol,mepaqi,neph
0,2020-03-19,01:00:00,Jieyang (揭阳),23.55,116.373,173,10,82,18.3,1.7,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-18,20:00:00,"Elazig, Turkey",38.6746,39.2117,24,NaN,93,NaN,NaN,...,1.5,5,12.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-18,11:00:00,"Arlington Municipal Airport, fort Worth, Dalla...",32.6564,-97.0886,33,NaN,76,2.5,12.8,...,5.2,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-07-13,06:00:00,"Tecnologico, Colima, Mexico",19.2621,-103.724,-,36.9,NaN,1.1,0.8,...,6.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-18,12:00:00,"Coast Guard Station - Davis Island, Hillsborou...",27.9283,-82.4544,29,1.5,62,1.2,15.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,2020-03-17,22:00:00,"Benito Juárez, México, Mexico",19.3716,-99.159,74,11.2,45,29.7,11.2,...,3,6,14.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
710,2020-03-18,13:00:00,"Parque D.Pedro II, São Paulo, Brazil",-23.5448,-46.6277,63,1.9,55.5,13.3,46.8,...,0.5,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
711,2020-03-19,00:00:00,Shanghai (上海),31.2047,121.449,119,5.5,64,17.9,44.7,...,0.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,2020-03-18,21:00:00,"New Delhi US Embassy, India (नई दिल्ली अमेरिकी...",28.6358,77.2245,132,NaN,39,NaN,NaN,...,5.1,13,10.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
